In [1]:
import os
import logging
import string
import sentencepiece
import pandas as pd
import textwrap
import PyPDF2
import matplotlib.pyplot as plt
import spacy

from transformers import pipeline
from transformers import AutoTokenizer, AutoModel
from spacy.language import Language
from pdfminer.high_level import extract_text

In [2]:
PROJECT_DIR = os.path.dirname(os.getcwd()) + '/mesure/'
file = PROJECT_DIR + '/1709.06206.pdf'
logging.getLogger('pdfminer').setLevel(logging.WARNING)

In [3]:
wrapper = textwrap.TextWrapper(width=80, break_long_words=False, break_on_hyphens=False)

In [4]:
def extract_text_from_pdf(file):
    text = extract_text(file)
    text = text.replace("\n", " ")
    text = text.replace("\x00", " ")
    text = ' '.join(text.split())
    return text

text = extract_text_from_pdf(file)

In [5]:
print(wrapper.fill(text[:700]))

Algorithm and Hardware Design of Discrete-Time Spiking Neural Networks Based on
Back Propagation with Binary Activations Shihui Yin*, Shreyas K.
Venkataramanaiah*, Gregory K. Chen†, Ram Krishnamurthy†, Yu Cao*, Chaitali
Chakrabarti*, Jae-sun Seo* *School of Electrical, Computer and Energy
Engineering, Arizona State University, Tempe, AZ, USA †Circuits Research Lab,
Intel Corporation, Hillsboro, OR, USA Email: syin11@asu.edu the difficulty
Abstract—We present a new back propagation based training algorithm for
discrete-time spiking neural networks (SNN). Inspired by recent deep learning
algorithms on binarized neural networks, binary activation with a
straight-through gradient estimator is us


# Question answering

In [6]:
config = {
    'task': 'question-answering',
    'tokenizer': 'distilbert-base-cased-distilled-squad',
    'model': 'distilbert-base-cased-distilled-squad'
}

qa_pipeline = pipeline(**config)

In [7]:
question = "What is the dataset?"

outputs = qa_pipeline(question=question, context=text, top_k=5)
outputs_small = [{'answer': pred['answer'], 'score': f'{pred["score"]:.2f}'} for pred in outputs]

/nix/store/irbwr09j59kv3rwid140ncihaqli8x32-python3-3.8.11-env/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:705: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)
/nix/store/irbwr09j59kv3rwid140ncihaqli8x32-python3-3.8.11-env/lib/python3.8/site-packages/transformers/pipelines/question_answering.py:295: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  p_mask = np.asarray(


In [8]:
outputs_small

[{'answer': 'saccade-1', 'score': '0.64'},
 {'answer': 'static image datasets', 'score': '0.52'},
 {'answer': 'N-MNIST', 'score': '0.50'},
 {'answer': 'N-MNIST', 'score': '0.35'},
 {'answer': 'spike sparsity', 'score': '0.28'}]

# Keywords extraction

## BERT Tokenizer

In [9]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
tokenizer.tokenize(text)[:15]

['algorithm',
 'and',
 'hardware',
 'design',
 'of',
 'discrete',
 '-',
 'time',
 'sp',
 '##iki',
 '##ng',
 'neural',
 'networks',
 'based',
 'on']

## Spacy

In [10]:
nlp = spacy.load("en_core_web_sm")
doc = nlp(text)

In [11]:
remove_words = {'Fig', 'INTRODUCTION', 'Abstract', 'Section'}
pos_to_keep = {'NOUN', 'ADJ', 'PROPN'}

def keep_token(token):
    # Remove all but nouns, adjectives and proper noun
    if token.pos_ not in pos_to_keep:
        return False
    
    # Remove punctuation
    if token.text in string.punctuation:
        return False
    
    # Remove units, math signs, 2-char strings might not be relevant
    if len(token.text) <= 2:
        return False
    
    # Remove custom words
    if token.text in remove_words:
        return False
    
    # Remove numbers (including numbers like 1e-5)
    if any(map(str.isdigit, token.text)):
        return False
    

    return True

tokens = [token for token in list(filter(keep_token, doc))]

In [12]:
tokens[100:140]

[MNIST,
 low,
 energy,
 image,
 Keywords,
 neural,
 networks,
 propagation,
 neuromorphic,
 hardware,
 straight,
 estimator,
 many,
 deep,
 learning,
 algorithms,
 such,
 multi,
 layer,
 perceptrons,
 MLP,
 convolutional,
 neural,
 networks,
 CNN,
 human,
 level,
 recognition,
 accuracy,
 image,
 speech,
 classification,
 tasks,
 number,
 operations,
 artificial,
 neural,
 networks,
 ANN,
 input]